In [37]:
using DataFrames, CSV, Plots, Random, DecisionTree, Statistics, Impute

┌ Info: Precompiling Impute [f7bf1975-0170-51b9-8c5f-a992d46b9575]
└ @ Base loading.jl:1278


In [2]:
using ScikitLearn.CrossValidation: train_test_split

In [3]:
function score_core_dataset(X_train, X_valid, y_train, y_valid)
    rf = RandomForestClassifier();
    DecisionTree.fit!(rf, X_train, y_train)
    pred = DecisionTree.predict(rf, X_valid);
    return mean_absolute_error(pred-y_valid)
end 

score_core_dataset (generic function with 1 method)

In [4]:
housing_prices_train_full = CSV.read("//home//barabasz//src//learning-julia//Housing Prices Competition//train.csv", DataFrame; types=Dict(:LotFrontage=>Int, :MasVnrArea=>Int, :GarageYrBlt=>Int), silencewarnings=true);
housing_prices_test_full = CSV.read("//home//barabasz//src//learning-julia//Housing Prices Competition//test.csv", DataFrame; types=Dict(:LotFrontage=>Int, :MasVnrArea=>Int, :GarageYrBlt=>Int), silencewarnings=true);
println( "size of training set $(size(housing_prices_train_full)[1])")
println( "size of testing set $(size(housing_prices_test_full)[1])")

size of training set 1460
size of testing set 1459


In [92]:
mapcols(x->eltype(x),housing_prices_test_full)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley
,DataType,DataType,DataType,Union,DataType,DataType,DataType
1,Int64,Int64,String,"Union{Missing, Int64}",Int64,String,String


In [94]:
nonmissingtype( eltype(housing_prices_test_full.LotFrontage) )

Int64

In [5]:
ms = mapcols(x->sum(ismissing,x),housing_prices_train_full)
ms=ms[!, convert(Vector,mapcols(x-> x.>0, ms)[1,:])]
#df[!,mapcols(x -> sum(ismissing, x), df)]
println( "total missing values ",sum(ms[1,:]) )
ms

total missing values 348


,LotFrontage,MasVnrArea,GarageYrBlt
,Int64,Int64,Int64
1,259,8,81


In [6]:
#filtered = convert( Array, data[!,[:Rooms, :Bathroom, :Landsize, :BuildingArea, :YearBuilt, :Lattitude, :Longtitude]] );
#y = convert( Array, data[!,:Price] );

y = convert(Array, housing_prices_train_full.SalePrice);
#features = [:LotArea, :YearBuilt, :1stFlrSF, :2ndFlrSF, :FullBath, :BedroomAbvGr, :TotRmsAbvGrd]
features = ["LotArea", "YearBuilt", "1stFlrSF", "2ndFlrSF", "FullBath", "BedroomAbvGr", "TotRmsAbvGrd"];

X_train_full = convert(Array, housing_prices_train_full[!,features]);
X_test  = convert(Array,  housing_prices_test_full[!,features]);

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y, train_size=0.8, test_size=0.2, random_state=0);
println("size of X_train $(size(X_train))")
println("size of X_valid $(size(X_valid))")
println("size of y_train $(size(y_train))")
println("size of y_valid $(size(y_valid))")

size of X_train (1168, 7)
size of X_valid (292, 7)
size of y_train (1168,)
size of y_valid (292,)


In [7]:
X_train[1:10,:]

10×7 Array{Int64,2}:
 11694  2007  1828    0  2  3  9
  6600  1962   894    0  1  2  5
 13360  1921   964    0  1  2  5
 13265  2002  1689    0  2  3  7
 13704  2001  1541    0  2  3  6
  7500  2004  1221    0  2  2  6
  8658  1965  1324    0  2  3  6
  2572  1999   696  720  2  3  6
  1596  1973   526  462  1  2  5
  6000  1941   869  349  1  3  6

# DataFrames play with missing values

In [8]:
df = DataFrame([5:7, [1, missing,3] ,[2, missing,missing]])

,x1,x2,x3
,Int64,Int64?,Int64?
1,5,1,2
2,6,missing,missing
3,7,3,missing


In [9]:
#mapcols(x -> any(ismissing, x), df)
c=mapcols(x -> any(ismissing.(x)), df)

,x1,x2,x3
,Bool,Bool,Bool
1,0,1,1


# completecases return a bit vector with 1 for each row without missing values

In [10]:
completecases(df)

3-element BitArray{1}:
 1
 0
 0

In [11]:
cols=convert(Vector, mapcols(x -> !any(ismissing.(x)), df)[1,:])
#cols=convert(Vector, mapcols(x -> sum(ismissing,x)>0, df)[1,:])
df[!,cols]
#df[!,[true false false]]
#[true false false]
#[1,2,3]
#[1 2 3]

,x1
,Int64
1,5
2,6
3,7


In [12]:
#colwise(x -> sum(ismissing.(x)), df) colwise is deprecated
#mapcols(x -> sum(ismissing.(x)), df)
df[!,mapcols(x -> sum(ismissing, x), df)]

LoadError: MethodError: no method matching getindex(::DataFrame, ::typeof(!), ::DataFrame)
Closest candidates are:
  getindex(::DataFrame, ::typeof(!), !Matched::Union{Signed, Unsigned}) at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/dataframe/dataframe.jl:431
  getindex(::DataFrame, ::typeof(!), !Matched::Union{AbstractString, Symbol}) at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/dataframe/dataframe.jl:439
  getindex(::DataFrame, ::typeof(!), !Matched::Union{Colon, Regex, AbstractArray{T,1} where T, All, Between, Cols, InvertedIndex}) at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/dataframe/dataframe.jl:478
  ...

In [156]:
filename_long = "//home//barabasz//src//learning-julia//Data//melb-house-prices-train.csv"
data = CSV.read("Data//melb-house-prices-train.csv", DataFrame; types=Dict(:LotFrontage=>Int, :MasVnrArea=>Int, :GarageYrBlt=>Int), silencewarnings=true);

In [14]:
sum_missing = mapcols(x -> sum(ismissing, x), data)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,259,0,0,0,0,0


In [15]:
#data[!, convert(Vector,(sum_missing .> )[1,:])]
sum_missing[sum_missing .> 0]
#(sum_missing .> 0)[1,:]
#AsTable(sum_missing .>0)

LoadError: MethodError: no method matching getindex(::DataFrame, ::DataFrame)
Closest candidates are:
  getindex(::DataFrame, !Matched::Integer, !Matched::Union{Signed, Unsigned}) at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/dataframe/dataframe.jl:388
  getindex(::DataFrame, !Matched::Integer, !Matched::Union{AbstractString, Symbol}) at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/dataframe/dataframe.jl:403
  getindex(::DataFrame, !Matched::AbstractArray{T,1} where T, !Matched::Union{AbstractString, Signed, Symbol, Unsigned}) at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/dataframe/dataframe.jl:413
  ...

In [16]:
nt = DataFrame(name = names(data), type=eltype.(eachcol(data)));
first(nt,5)

,name,type
,String,Type
1,Id,Int64
2,MSSubClass,Int64
3,MSZoning,String
4,LotFrontage,"Union{Missing, Int64}"
5,LotArea,Int64


In [17]:
ba = (<:).(nt[!,:type],Union{Missing,Number})
println(size(ba))
ba[1:6]

(81,)


6-element BitArray{1}:
 1
 1
 0
 1
 1
 0

In [18]:
Int <: Union{Missing,Number}, Union{Missing,Number} <: Int, Union{Missing,Number} <: Union{Missing,Number}, String<:Union{Missing,Number}

(true, false, true, false)

In [19]:
Int <: Number, Number <: Int

(true, false)

In [20]:
filter(row -> row.type == Union{Missing,Int64}, nt);
nt[nt.type .== Union{Missing,Int64}, :]

,name,type
,String,Type
1,LotFrontage,"Union{Missing, Int64}"
2,MasVnrArea,"Union{Missing, Int64}"
3,GarageYrBlt,"Union{Missing, Int64}"


In [21]:
methods(select)

# 4 methods for generic function "select":
[1] select(df::AbstractDataFrame, args...; copycols, renamecols) in DataFrames at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/abstractdataframe/selection.jl:847
[2] select(arg::Union{Function, Type}, df::AbstractDataFrame; renamecols) in DataFrames at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/abstractdataframe/selection.jl:850
[3] select(f::Union{Function, Type}, gd::GroupedDataFrame; copycols, keepkeys, ungroup, renamecols) in DataFrames at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/groupeddataframe/splitapplycombine.jl:642
[4] select(gd::GroupedDataFrame, args...; copycols, keepkeys, ungroup, renamecols) in DataFrames at /home/barabasz/.julia/packages/DataFrames/oQ5c7/src/groupeddataframe/splitapplycombine.jl:652

In [22]:
@time transpose(reshape(1:24,4,6))

  0.000000 seconds


6×4 LinearAlgebra.Transpose{Int64,Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}}}:
  1   2   3   4
  5   6   7   8
  9  10  11  12
 13  14  15  16
 17  18  19  20
 21  22  23  24

In [23]:
@time reshape(1:24,4,6) |> transpose

  0.000000 seconds


6×4 LinearAlgebra.Transpose{Int64,Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}}}:
  1   2   3   4
  5   6   7   8
  9  10  11  12
 13  14  15  16
 17  18  19  20
 21  22  23  24

In [24]:
a=[r*4+c for r in 0:5,c in 1:4]

6×4 Array{Int64,2}:
  1   2   3   4
  5   6   7   8
  9  10  11  12
 13  14  15  16
 17  18  19  20
 21  22  23  24

In [25]:
df2 = DataFrame(a);
(df2 .>= 10) .& (df2 .< 18)

,x1,x2,x3,x4
,Bool,Bool,Bool,Bool
1,0,0,0,0
2,0,0,0,0
3,0,1,1,1
4,1,1,1,1
5,1,0,0,0
6,0,0,0,0


In [26]:
s=rand(Bool,6,4)
println(s)
#df2[s] = missing
df3 = DataFrame( [s[r,c] ? df2[r,c] : missing for r in 1:6,c in 1:4])
df3

Bool[1 0 1 1; 0 1 1 0; 0 1 0 1; 0 1 1 1; 1 0 1 0; 0 0 0 1]


,x1,x2,x3,x4
,Int64?,Int64?,Int64?,Int64?
1,1,missing,3,4
2,missing,6,7,missing
3,missing,10,missing,12
4,missing,14,15,16
5,17,missing,19,missing
6,missing,missing,missing,24


In [61]:
#disallowmissing(df3) ->
#MethodError: Cannot `convert` an object of type Missing to an object of type Int64

# fill missing values with a specified value

In [27]:
coalesce.(df3,-1)

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,-1,3,4
2,-1,6,7,-1
3,-1,10,-1,12
4,-1,14,15,16
5,17,-1,19,-1
6,-1,-1,-1,24


In [51]:
mean_by_column = mapcols(x->mean(skipmissing(x)), df3)
df4 = copy(df3)
for cn in names(df4)
    replace!(df4[!,cn], missing=>mean_by_column[1,cn])
end

,x1,x2,x3,x4
,Int64?,Int64?,Int64?,Int64?
1,1,10,3,4
2,9,6,7,14
3,9,10,11,12
4,9,14,15,16
5,17,10,19,14
6,9,10,11,24


In [83]:
df4 = copy(df3)
@time for cn in names(df4)
    col = df4[!,cn]
    this_column_mean = mean(skipmissing(col))
    #replace!(col, missing=>this_column_mean)
    #coalesce.(col, this_column_mean)
    col[ ismissing.(col)] .= this_column_mean
end
disallowmissing(df4)

  0.000050 seconds (61 allocations: 2.516 KiB)


,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,10,3,4
2,9,6,7,14
3,9,10,11,12
4,9,14,15,16
5,17,10,19,14
6,9,10,11,24


In [86]:
#mapcols(x->count(ismissing.(x)), df3)
#mapcols(x->count(ismissing, x), df3)
mapcols(x->sum(ismissing, x), df3)

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,4,3,2,2


## using Impute package
## https://juliahub.com/docs/Impute/tKYqZ/0.6.5/autodocs/

In [63]:
Impute.interp(df3)

,x1,x2,x3,x4
,Int64?,Int64?,Int64?,Int64?
1,1,missing,3,4
2,5,6,7,8
3,9,10,11,12
4,13,14,15,16
5,17,missing,19,20
6,missing,missing,missing,24


In [64]:
#Iterates forwards through the data and fills missing data with the last existing observation
Impute.locf(df3)

,x1,x2,x3,x4
,Int64?,Int64?,Int64?,Int64?
1,1,missing,3,4
2,1,6,7,4
3,1,10,7,12
4,1,14,15,16
5,17,14,19,16
6,17,14,19,24


In [66]:
println(df3)
#Iterates backwards through the data and fills missing data with the next existing observation
Impute.nocb(df3)

6×4 DataFrame
 Row │ x1       x2       x3       x4      
     │ Int64?   Int64?   Int64?   Int64?  
─────┼────────────────────────────────────
   1 │       1  missing        3        4
   2 │ missing        6        7  missing 
   3 │ missing       10  missing       12
   4 │ missing       14       15       16
   5 │      17  missing       19  missing 
   6 │ missing  missing  missing       24


,x1,x2,x3,x4
,Int64?,Int64?,Int64?,Int64?
1,1,6,3,4
2,17,6,7,12
3,17,10,15,12
4,17,14,15,16
5,17,missing,19,24
6,missing,missing,missing,24


In [88]:
Impute.interp(df3) |> Impute.locf() |> Impute.nocb()

,x1,x2,x3,x4
,Int64?,Int64?,Int64?,Int64?
1,1,6,3,4
2,5,6,7,8
3,9,10,11,12
4,13,14,15,16
5,17,14,19,20
6,17,14,19,24


In [118]:
#Simple Random Sampling (SRS) imputation is a method for imputing both continuous and categorical variables. 
#Furthermore, it completes imputation while preserving the distributional properties of the variables 
#(e.g., mean, standard deviation).
Impute.srs(df3)

,x1,x2,x3,x4
,Int64?,Int64?,Int64?,Int64?
1,1,10,3,4
2,17,6,7,12
3,17,10,3,12
4,17,14,15,16
5,17,14,19,16
6,17,6,7,24


In [122]:
Impute.substitute(df3;statistic=median)

,x1,x2,x3,x4
,Int64?,Int64?,Int64?,Int64?
1,1,10,3,4
2,9,6,7,14
3,9,10,11,12
4,9,14,15,16
5,17,10,19,14
6,9,10,11,24


In [108]:
#Finds variables with missing values in a AbstractMatrix or Tables.table and removes them from the input data
Impute.dropvars(df3)

""
""


In [110]:
Impute.filter(df3; dims=:cols)
Impute.filter(df3; dims=:rows)

""
""


In [113]:
using NearestNeighbors

In [130]:
#Impute.knn(df3; k=1, threshold=0.5, dist=Euclidean())
#Impute.knn(df3; dims=:cols)
#Impute.knn(df3; k=1)

In [28]:
replace(df3.x1, missing=>NaN)

6-element Array{Float64,1}:
   1.0
 NaN
 NaN
 NaN
  17.0
 NaN

In [29]:
df1 = DataFrame(name=["John", "Sally", "Kirk"], 
               age=[23., 42., 59.],
               children=[3,5,2], height = [180, 150, 170])

,name,age,children,height
,String,Float64,Int64,Int64
1,John,23.0,3,180
2,Sally,42.0,5,150
3,Kirk,59.0,2,170


In [30]:
mapcols(x->mean(x),df1[!,Not(:name)])

,age,children,height
,Float64,Float64,Float64
1,41.3333,3.33333,166.667


In [31]:
df1."name"

3-element Array{String,1}:
 "John"
 "Sally"
 "Kirk"

In [32]:
#eltype.(df)
eltype.(eachcol(df1))

4-element Array{DataType,1}:
 String
 Float64
 Int64
 Int64

In [98]:
println( "count(df1.height .> 150) = ", count(df1.height .> 150) )
println( "sum(df1.children) = "       ,sum(df1.children) )
println( "sum(x -> x>2, df1.children) = ", sum(x -> x>2, df1.children))
println( "count(df1.children .> 2) = ", count(df1.children .> 2))

count(df1.height .> 150) = 2
sum(df1.children) = 10
sum(x -> x>2, df1.children) = 2
count(df1.children .> 2) = 2
